![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [92]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head(20)

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
5,21651,city_176,0.764,NaN,Has relevant experience,Part time course,Graduate,STEM,11,NaN,NaN,1,24,1.0
6,28806,city_160,0.920,Male,Has relevant experience,no_enrollment,High School,NaN,5,50-99,Funded Startup,1,24,0.0
7,402,city_46,0.762,Male,Has relevant experience,no_enrollment,Graduate,STEM,13,<10,Pvt Ltd,>4,18,1.0
8,27107,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,7,50-99,Pvt Ltd,1,46,1.0
9,699,city_103,0.920,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0.0


In [93]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

ds_jobs_transformed.info()

for col in ds_jobs_transformed.columns:
    print(f"\n\n{col}: {ds_jobs_transformed[col].nunique()} unique\n{ds_jobs_transformed[col].value_counts(dropna=False)}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [94]:
ds_jobs_transformed = ds_jobs.copy()


ds_jobs_transformed['student_id'] = ds_jobs_transformed['student_id'].astype(np.int32)

ds_jobs_transformed['city'] = ds_jobs_transformed['city'].astype('category')
city_sorted = sorted(ds_jobs_transformed['city'].cat.categories)
ds_jobs_transformed['city'].cat.reorder_categories(city_sorted, inplace=True)

ds_jobs_transformed['city_development_index'] = ds_jobs_transformed['city_development_index'].astype(np.float16)

ds_jobs_transformed['gender'] = ds_jobs_transformed['gender'].astype('category').cat.reorder_categories(['Male', 'Female', 'Other'], ordered=True)
ds_jobs_transformed['gender'] = ds_jobs_transformed['gender'].fillna('Other')

ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].astype('category')
ds_jobs_transformed['relevant_experience'] = (ds_jobs_transformed['relevant_experience'] == ds_jobs_transformed['relevant_experience'].unique()[0])

ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].astype('category').cat.set_categories(['Full time course', 'Part time course', 'Other'], ordered=True).fillna('Other')

ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].astype('category').cat.add_categories('Unknown').fillna('Unknown')\
.cat.reorder_categories(['Phd', 'Masters', 'Graduate', 'High School', 'Primary School', 'Unknown'], ordered=True)

ds_jobs_transformed['major_discipline'] = ds_jobs_transformed['major_discipline'].astype('category').cat.add_categories('Unknown').fillna('Unknown')

years = [str(i) for i in range(10, 21)] + ['>20']
ds_jobs_transformed['experience'] = ds_jobs_transformed['experience'].astype('category').cat.set_categories(years, ordered=True)
#print(ds_jobs_transformed['experience'].value_counts(dropna=False, sort=False))
ds_jobs_transformed = ds_jobs_transformed[ds_jobs_transformed['experience'].isna()==False]

company_size_ordered = ['1000-4999', '5000-9999', '10000+']
ds_jobs_transformed['company_size'] = pd.Categorical(ds_jobs_transformed['company_size'], 
                                    categories=company_size_ordered, 
                                    ordered=True)
ds_jobs_transformed = ds_jobs_transformed[ds_jobs_transformed['company_size'].isna()==False]

ds_jobs_transformed['company_type'] = ds_jobs_transformed['company_type'].astype('category').cat.add_categories('Unknown').fillna('Unknown')

new_job_ordered = ['1', '2', '3', '4', '>4', 'never', 'Unknown']
ds_jobs_transformed['last_new_job'] = pd.Categorical(ds_jobs_transformed['last_new_job'], 
                                    categories=new_job_ordered, 
                                    ordered=True).fillna('Unknown')
#print(ds_jobs_transformed['last_new_job'].value_counts(dropna=False, sort=False))

ds_jobs_transformed['training_hours'] = ds_jobs_transformed['training_hours'].astype(np.int32)

ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'].astype(bool)



ds_jobs_transformed.info()

print(ds_jobs_transformed.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  2201 non-null   category
 4   relevant_experience     2201 non-null   bool    
 5   enrolled_university     2201 non-null   category
 6   education_level         2201 non-null   category
 7   major_discipline        2201 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2201 non-null   category
 11  last_new_job            2201 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   bool    
dtypes: bool(2), category(9)